In [2]:
import veloxchem as vlx

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 8.


In [3]:
class MockTask:
    def __init__(self, mol, basis, comm, ostream):

        self.molecule = mol
        self.ao_basis = basis
        self.mpi_comm = comm
        self.ostream = ostream

In [4]:
import adcc
from mpi4py import MPI
import sys
import numpy
numpy.set_printoptions(precision=7, suppress=True)

In [173]:
# Input settings
molecule_string = """
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037"""

#molecule_string = """
#    H 0 0 0
#    H 0 0 1.795239827225189
#    """

basis_set_label = '6-31G'
scf_settings = {'conv_thresh': 1.0e-6}
method_settings = {} #{'xcfun': 'b3lyp', 'grid_level': 4}
rsp_settings = {'conv_thresh': 1.0e-4, 'nstates': 1}

In [6]:
# Communicator and output stream
comm = MPI.COMM_WORLD
ostream = vlx.OutputStream(sys.stdout)

In [174]:
# Molecule and basis set
molecule = vlx.Molecule.read_str(molecule_string, units='au')
basis = vlx.MolecularBasis.read(molecule, basis_set_label)

ostream.print_block(molecule.get_string())
ostream.print_block(basis.get_string('Atomic Basis', molecule))
ostream.flush()

                                              Molecular Geometry (Angstroms)                                              
                                                                                                                          
                          Atom         Coordinate X          Coordinate Y          Coordinate Z                           
                                                                                                                          
                           O           0.000000000000        0.000000000000        0.000000000000                         
                           H           0.000000000000        0.000000000000        0.950000004704                         
                           H           0.896000004436        0.000000000000       -0.317000001569                         
                                                                                                                          
                

In [175]:
# SCF
task = MockTask(molecule, basis, comm, ostream)
scfdrv = vlx.ScfRestrictedDriver(comm, ostream)
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(molecule, basis)
scfdrv.task = task

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [177]:
# SCF first-order properties
scf_prop = vlx.ScfProperties(comm, ostream)
scf_prop.compute(molecule, basis, scfdrv.scf_tensors)
scf_prop.print_properties(molecule)

                                                                                                                          
                                                Ground-State Dipole Moment                                                
                                               ----------------------------                                               
                                                                                                                          
                                   X   :         0.815852 a.u.         2.073688 Debye                                     
                                   Y   :         0.000000 a.u.         0.000000 Debye                                     
                                   Z   :         0.576635 a.u.         1.465659 Debye                                     
                                 Total :         0.999060 a.u.         2.539358 Debye                                     
                

In [10]:
# Linear Response
#lr_eig_drv = vlx.LinearResponseEigenSolver(comm, ostream)
#lr_eig_drv.update_settings(rsp_settings, method_settings)
#lr_results = lr_eig_drv.compute(molecule, basis, scfdrv.scf_tensors)
#lr_eig_drv.ostream.flush()

In [178]:
# TDHF/TDA, i.e. CIS
tda_drv = vlx.TDAExciDriver(comm, ostream)
tda_drv.update_settings(rsp_settings, method_settings)
tda_results = tda_drv.compute(molecule, basis, scfdrv.scf_tensors)
tda_drv.ostream.flush()

                                                                                                                          
                                                     TDA Driver Setup                                                     
                                                                                                                          
                               Number of States                : 1                                                        
                               Max. Number of Iterations       : 150                                                      
                               Convergence Threshold           : 1.0e-04                                                  
                               ERI Screening Scheme            : Cauchy Schwarz + Density                                 
                               ERI Screening Threshold         : 1.0e-15                                                  
                

In [179]:
# Get MO coefficients, eigenvalues and eigenvectors

nocc = molecule.number_of_alpha_electrons()
mo = scfdrv.scf_tensors['C'] # MO coefficients
mo_occ = mo[:, :nocc]        # occupied
mo_vir = mo[:, nocc:]        # virtual

nocc = mo_occ.shape[1]
nvir = mo_vir.shape[1]

#eig_vals = lr_results["eigenvalues"]
#eig_vecs = lr_results["eigenvectors"]

#eig_vec = eig_vecs[:, 0].copy() # first eigenvector
#print(eig_vec)
print("mo_occ:")
print(mo_occ)
print()
print("mo_vir:")
print(mo_vir)

mo_occ:
[[-0.9957718  0.2128936 -0.0000549 -0.072784   0.       ]
 [-0.0220092 -0.4697032  0.0001011  0.1737356 -0.       ]
 [ 0.0080636 -0.4789322  0.0003031  0.2951384 -0.       ]
 [-0.0000658 -0.1411472 -0.265331  -0.1371443  0.       ]
 [-0.0020096  0.0085744 -0.1168805 -0.0762051  0.       ]
 [-0.0000666 -0.1409672  0.2652191 -0.137267  -0.       ]
 [-0.0020072  0.0085556  0.1169106 -0.076352   0.       ]
 [ 0.        -0.        -0.        -0.        -0.6406288]
 [-0.        -0.        -0.        -0.        -0.5110702]
 [-0.0011822 -0.0624506 -0.4125042 -0.321957   0.       ]
 [ 0.0010617 -0.0332144 -0.2154389 -0.2389627  0.       ]
 [-0.001671  -0.0881485  0.2914149 -0.4556016  0.       ]
 [ 0.0015007 -0.0469772  0.1522514 -0.3381818  0.       ]]

mo_vir:
[[-0.0869358  0.0000554 -0.000131   0.         0.0702479 -0.043622
   0.0000626  0.059421 ]
 [ 0.1069212 -0.0000445  0.0007821 -0.        -0.3253915  0.2719421
  -0.0004268 -1.6971549]
 [ 1.2057539 -0.0011996 -0.0011375  0.     

In [180]:
print(scfdrv.scf_tensors.keys())
# Transform the Fock matrix from AO to MO basis
fock = scfdrv.scf_tensors['F'][0]
print(numpy.matmul(mo.T, numpy.matmul(fock, mo)))

dict_keys(['C', 'E', 'S', 'D', 'F'])
[[-20.5552214  -0.         -0.          0.         -0.          0.
   -0.         -0.          0.         -0.          0.         -0.
   -0.       ]
 [ -0.         -1.354532    0.         -0.         -0.         -0.
    0.         -0.         -0.          0.         -0.          0.
    0.0000001]
 [ -0.          0.         -0.7227064   0.          0.         -0.
   -0.0000001  -0.         -0.          0.          0.         -0.0000001
    0.       ]
 [  0.         -0.          0.         -0.5520806  -0.         -0.0000001
    0.          0.          0.         -0.          0.          0.
    0.       ]
 [ -0.         -0.          0.         -0.         -0.4993126   0.
    0.          0.         -0.         -0.         -0.          0.
    0.       ]
 [  0.         -0.         -0.         -0.0000001   0.          0.2065263
   -0.          0.          0.         -0.          0.         -0.
   -0.       ]
 [ -0.          0.         -0.0000001   0.      

In [13]:
# Convert the first eigenvector to matrix form
#eig_vec_as_mat = vlx.LinearResponseEigenSolver.lrvec2mat(eig_vec, nocc, nocc + nvir)

In [181]:
tda_eig_vals = tda_results["eigenvalues"]
tda_eig_vecs = tda_results["eigenvectors"]
tda_size = tda_eig_vecs[:,0].shape
tda_eig_vec=tda_eig_vecs[:,0].copy()
tda_eig_vec_as_mat = tda_eig_vec.reshape(nocc, nvir) #/ numpy.sqrt(2.0)

In [182]:
print(tda_eig_vec)

[ 0.        -0.         0.        -0.0006095  0.        -0.
  0.         0.         0.         0.        -0.         0.0020951
  0.         0.        -0.        -0.        -0.        -0.
 -0.         0.0000019  0.         0.        -0.        -0.
 -0.        -0.         0.        -0.0045535  0.        -0.
  0.        -0.         0.992417   0.000373   0.0003748 -0.
 -0.0906023 -0.0605484  0.0000866 -0.0566383]


In [16]:
#tda_eig_vec_as_mat = vlx.LinearResponseEigenSolver.lrvec2mat(tda_eig_vec, nocc, nocc + nvir)

In [183]:
print(tda_eig_vec_as_mat)

[[ 0.        -0.         0.        -0.0006095  0.        -0.
   0.         0.       ]
 [ 0.         0.        -0.         0.0020951  0.         0.
  -0.        -0.       ]
 [-0.        -0.        -0.         0.0000019  0.         0.
  -0.        -0.       ]
 [-0.        -0.         0.        -0.0045535  0.        -0.
   0.        -0.       ]
 [ 0.992417   0.000373   0.0003748 -0.        -0.0906023 -0.0605484
   0.0000866 -0.0566383]]


In [184]:
adc1 = adcc.run_adc(scfdrv, method='adc1', n_singlets=1, conv_tol=1e-4)

Starting adc1 singlet Jacobi-Davidson ...
Niter n_ss  max_residual  time  Ritz values
  1     4      0.013011   10ms  [0.3628166]
  2     8    9.7587e-06   15ms  [0.3491059]
=== Converged ===
    Number of matrix applies:    8
    Total solver time:           28.097ms


In [210]:
adc1_eig_vec = adc1.excitation_vector[0]["ph"].to_ndarray()
adc1_eig_vals = adc1.excitation_energy
adc1.state_dipole_moment

array([[-0.2186835,  0.       , -0.1497238]])

In [186]:
print(adc1_eig_vec)
print(adc1_eig_vals)
print(adc1.describe())

[[ 0.         0.         0.         0.0003757 -0.         0.
   0.        -0.         0.         0.         0.         0.
   0.         0.         0.         0.       ]
 [-0.         0.        -0.        -0.0016012  0.        -0.
  -0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.       ]
 [-0.         0.         0.        -0.0000019 -0.        -0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.       ]
 [-0.         0.         0.         0.0028569 -0.         0.
   0.        -0.         0.         0.         0.         0.
   0.         0.         0.         0.       ]
 [-0.7017908 -0.0004388 -0.000281   0.         0.0639281  0.0412832
  -0.0000922  0.0410785  0.         0.         0.         0.
   0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.0003757
  -0.         0.   

In [187]:
print(tda_eig_vecs[:,0])
print(tda_eig_vals)

[ 0.        -0.         0.        -0.0006095  0.        -0.
  0.         0.         0.         0.        -0.         0.0020951
  0.         0.        -0.        -0.        -0.        -0.
 -0.         0.0000019  0.         0.        -0.        -0.
 -0.        -0.         0.        -0.0045535  0.        -0.
  0.        -0.         0.992417   0.000373   0.0003748 -0.
 -0.0906023 -0.0605484  0.0000866 -0.0566383]
[0.3490989]


In [188]:
numpy.linalg.norm(adc1_eig_vec)

0.9999999999999999

In [189]:
0.9999995/numpy.sqrt(2.0)

0.7071064276331569

In [190]:
0.0009539/numpy.sqrt(2.0)

0.0006745091585738476

In [191]:
adc1_eig_vec

array([[ 0.       ,  0.       ,  0.       ,  0.0003757, -0.       ,
         0.       ,  0.       , -0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ],
       [-0.       ,  0.       , -0.       , -0.0016012,  0.       ,
        -0.       , -0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ],
       [-0.       ,  0.       ,  0.       , -0.0000019, -0.       ,
        -0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ],
       [-0.       ,  0.       ,  0.       ,  0.0028569, -0.       ,
         0.       ,  0.       , -0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ],
       [-0.7017908, -0.0004388, -0.000281 ,  0.       ,  0.0639281,
         0.0412832, -0.0000922, 

In [192]:
vlx_dm_oo = -numpy.einsum('ia,ja->ij', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_dm_vv = numpy.einsum('ia,ib->ab', tda_eig_vec_as_mat, tda_eig_vec_as_mat)
vlx_DM_ovov = -numpy.einsum('ib,ja->iajb', tda_eig_vec_as_mat, tda_eig_vec_as_mat)

print(vlx_dm_oo)

[[-0.0000004  0.0000013  0.        -0.0000028 -0.       ]
 [ 0.0000013 -0.0000044 -0.         0.0000095 -0.       ]
 [ 0.        -0.        -0.         0.         0.       ]
 [-0.0000028  0.0000095  0.        -0.0000207  0.       ]
 [-0.        -0.         0.         0.        -0.9999745]]


In [193]:
adcc_dm_oo = -numpy.einsum('ia,ja->ij', adc1_eig_vec, adc1_eig_vec)
adcc_dm_vv = numpy.einsum('ia,ib->ab', adc1_eig_vec, adc1_eig_vec)
adcc_DM_ovov = -numpy.einsum('ib,ja->iajb', adc1_eig_vec, adc1_eig_vec)

print(adcc_dm_oo)

foo = adc1.reference_state.fock("o1o1").to_ndarray()
fvv = adc1.reference_state.fock("v1v1").to_ndarray()
ovov = adc1.reference_state.eri("o1v1o1v1").to_ndarray()

adcc_excitation_energy = (numpy.einsum('ij,ij->',adcc_dm_oo,foo)
                     + numpy.einsum('ab,ab->', adcc_dm_vv, fvv)
                     + numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
                    )

[[-0.0000001  0.0000006  0.        -0.0000011  0.        -0.
  -0.        -0.        -0.        -0.       ]
 [ 0.0000006 -0.0000026 -0.         0.0000046 -0.        -0.
  -0.        -0.        -0.        -0.       ]
 [ 0.        -0.        -0.         0.        -0.        -0.
  -0.        -0.        -0.        -0.       ]
 [-0.0000011  0.0000046  0.        -0.0000082 -0.        -0.
  -0.        -0.        -0.        -0.       ]
 [ 0.        -0.        -0.        -0.        -0.4999891 -0.
  -0.        -0.        -0.        -0.       ]
 [-0.        -0.        -0.        -0.        -0.        -0.0000001
   0.0000006  0.        -0.0000011  0.       ]
 [-0.        -0.        -0.        -0.        -0.         0.0000006
  -0.0000026 -0.         0.0000046 -0.       ]
 [-0.        -0.        -0.        -0.        -0.         0.
  -0.        -0.         0.        -0.       ]
 [-0.        -0.        -0.        -0.        -0.        -0.0000011
   0.0000046  0.        -0.0000082 -0.       ]
 [-0.  

In [194]:
print(adcc_excitation_energy)

0.34910585522553944


$E = \frac{1}{4} \sum_{pqrs} \Gamma_{pqrs} \langle pq || rs \rangle$

$\langle ia || jb \rangle = - \langle ai || jb \rangle = - \langle ia || bj \rangle = \langle ai || bj \rangle$

analogous symmetry for $\boldsymbol{\Gamma}$

In [195]:
vlx_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +1.0*(
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        #+0.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, nocc:, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, nocc:, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, :nocc, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, :nocc, nvir:])# zero block
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir])##bbbb
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, :nocc, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, :nocc, nvir:])##abba
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, :nvir, nocc:, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, :nocc, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, :nocc, nvir:])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, nocc:, :nvir])# zero block
                        #+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[nocc:, nvir:, nocc:, nvir:])##baab
                        ))
print("vlx energy =", vlx_excitation_energy)

vlx energy = 0.3490988919255511


In [196]:
print(vlx_excitation_energy)

0.3490988919255511


In [197]:
adcc_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +2.0*numpy.einsum('iajb,iajb->', adcc_DM_ovov[:nocc, :nvir, :nocc, :nvir], ovov[:nocc, :nvir, :nocc, :nvir])
                        +2.0*numpy.einsum('iajb,iajb->', adcc_DM_ovov[:nocc, :nvir, :nocc, :nvir], ovov[:nocc, nvir:, nocc:, :nvir])
    )
print(adcc_excitation_energy)

0.34915590054854834


In [198]:
print(ovov.shape)
print(ovov)

(10, 16, 10, 16)
[[[[ 0.4371492  0.000079   0.0005805 ...  0.         0.
     0.       ]
   [-0.0032837 -0.0000088 -0.0000227 ...  0.         0.
     0.       ]
   [-0.0000047 -0.0031069  0.000645  ...  0.         0.
     0.       ]
   ...
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]]

  [[ 0.000079   0.4613466  0.1139556 ...  0.         0.
     0.       ]
   [-0.0000094 -0.0099228 -0.0043083 ...  0.         0.
     0.       ]
   [-0.0076622  0.0000091 -0.0000132 ...  0.         0.
     0.       ]
   ...
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]
   [ 0.         0.         0.        ...  0.         0.
     0.       ]]

  [[ 0.0005805  0.1139556  0.5087421 ...  0.         0.
     0.       ]
   [-0.0000165 -0.0034988 -0.

In [199]:
print(vlx_DM_ovov)
print()
print(vlx_DM_ovov.shape)
#print(adcc_DM_ovov)

[[[[-0.         0.        -0.        ...  0.        -0.
    -0.       ]
   [-0.         0.        -0.        ...  0.        -0.
    -0.       ]
   [ 0.        -0.         0.        ... -0.         0.
     0.       ]
   [ 0.        -0.         0.        ... -0.         0.
     0.       ]
   [-0.         0.        -0.        ...  0.        -0.
    -0.       ]]

  [[ 0.        -0.         0.        ... -0.         0.
     0.       ]
   [-0.         0.        -0.        ...  0.        -0.
    -0.       ]
   [ 0.        -0.         0.        ... -0.         0.
     0.       ]
   [ 0.        -0.         0.        ... -0.         0.
     0.       ]
   [-0.         0.        -0.        ...  0.        -0.
    -0.       ]]

  [[-0.         0.        -0.        ...  0.        -0.
    -0.       ]
   [ 0.        -0.         0.        ... -0.         0.
     0.       ]
   [ 0.        -0.         0.        ... -0.         0.
     0.       ]
   [-0.         0.        -0.        ...  0.        -0.
    

In [82]:
# Making the Vlx 2PDM twice the size with the other spin blocks
# and assign the corresponding values to it by comparing to the adcc matrix
# afterwards divide by two to yield the same numbers
vlx_DM_ovov_full = numpy.zeros((2*nocc, 2*nvir, 2*nocc, 2*nvir))
#vlx_DM_ovov_full[:nocc, :nvir, :nocc, :nvir] = vlx_DM_ovov
vlx_DM_ovov_full[0, 0, 0, 0] = vlx_DM_ovov_full[0, 3, 1, 0] = vlx_DM_ovov_full[1,0,0,3] = vlx_DM_ovov_full[1,3,1,3] = vlx_DM_ovov[0,0,0,0]
vlx_DM_ovov_full[0, 0, 0, 2] = vlx_DM_ovov_full[0, 2, 0, 0] = vlx_DM_ovov_full[0,3,1,2] = vlx_DM_ovov_full[0,5,1,0] = vlx_DM_ovov[0,2,0,0]
vlx_DM_ovov_full[1, 0, 0, 5] = vlx_DM_ovov_full[1, 2, 0, 3] = vlx_DM_ovov_full[1,3,1,5] = vlx_DM_ovov_full[1,5,1,3] = vlx_DM_ovov[0,2,0,0]
vlx_DM_ovov_full[0, 2, 0, 2] = vlx_DM_ovov_full[0, 5, 1, 2] = vlx_DM_ovov_full[1,2,0,5] = vlx_DM_ovov_full[1,5,1,5] = vlx_DM_ovov[0,2,0,2]

# matrix elements were twice as large
vlx_DM_ovov_full *= 0.5
# norm of the difference between the two density matrices does not seem to vanish entirely...
print("norm of the difference of the 2PDMs = ", numpy.linalg.norm(adcc_DM_ovov - vlx_DM_ovov_full))
#print(vlx_DM_ovov_full)
vlx_excitation_energy = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo,foo[:nocc,:nocc])
                        #(1.0*numpy.einsum('ij,ij->', adcc_dm_oo,foo)
                        #+1.0*numpy.einsum('ij,ij->', adcc_dm_vv,fvv)
                        +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir])
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov_full, ovov)
                        )
print("vlx exc. energy =", vlx_excitation_energy)
#print("adcc - vlx =", adcc_excitation_energy - vlx_excitation_energy)

norm of the difference of the 2PDMs =  2.6113194510212303e-08
vlx exc. energy = 0.4843494013574051
adcc - vlx = -3.885780586188048e-16


In [113]:
print(adcc_DM_ovov.shape)
print()
#print(adcc_DM_ovov)
print()
#print(adcc_DM_ovov - vlx_DM_ovov_full) # here everything appears to be zero

(2, 6, 2, 6)




In [200]:
# Transform the one-particle density matrices from MO to AO basis
vlx_dm_oo_in_ao = numpy.matmul(mo_occ, numpy.matmul(vlx_dm_oo, mo_occ.T))
vlx_dm_vv_in_ao = numpy.matmul(mo_vir, numpy.matmul(vlx_dm_vv, mo_vir.T))
vlx_dm_ao = vlx_dm_oo_in_ao + vlx_dm_vv_in_ao

contract_fock_mo = (1.0*numpy.einsum('ij,ij->', vlx_dm_oo, foo[:nocc,:nocc])
                    +1.0*numpy.einsum('ab,ab->', vlx_dm_vv, fvv[:nvir,:nvir]))

contract_fock_ao = numpy.einsum('mn,mn->', vlx_dm_ao, fock)

print("contract_fock_ao =", contract_fock_ao)
print("contract_fock_mo =", contract_fock_mo)

contract_fock_ao = 0.7222093146343266
contract_fock_mo = 0.7222093146343264


In [215]:
# Calculate the total density (including HF part)
# and calculated the unrelaxed excited-state dipole moment

# dipole integrals
dipole_drv = vlx.ElectricDipoleIntegralsDriver(comm)
dipole_mats = dipole_drv.compute(molecule, basis)
dipole_ints = (dipole_mats.x_to_numpy(),
              dipole_mats.y_to_numpy(),
              dipole_mats.z_to_numpy())

# electronic contribution
total_density = 2*scfdrv.scf_tensors['D'][0] + vlx_dm_ao
electronic_dipole = -1.0 * numpy.array(
    [numpy.sum(dipole_ints[d] * total_density) for d in range(3)])

# nuclear contribution
coords = molecule.get_coordinates()
nuclear_charges = molecule.elem_ids_to_numpy()
nuclear_dipole = numpy.sum(coords.T * nuclear_charges, axis=1)

dipole_moment = (nuclear_dipole + electronic_dipole)

print("dipole moment [au] =", dipole_moment)
print()
print("ADC(1) dipole moment for water in Q-Chem:")
print("Dip. moment [a.u.]:  [0.221200,  -0.00000,   0.151505]")
print()
print("ADC(1) dipole moment from adcc:")
print(adc1.state_dipole_moment)

dipole moment [au] = [-0.2206914  0.        -0.1522328]

ADC(1) dipole moment for water in Q-Chem:
Dip. moment [a.u.]:  [0.221200,  -0.00000,   0.151505]

ADC(1) dipole moment from adcc:
[[-0.2186835  0.        -0.1497238]]


In [119]:
# Transform the excitation vectors from MO to AO basis
tda_eig_vec_ao = numpy.matmul(mo_occ, numpy.matmul(tda_eig_vec_as_mat, mo_vir.T))
print(tda_eig_vec_ao)

[[ 0.0396684  0.3919179 -0.0396684 -0.3919179]
 [ 0.0432528  0.4273312 -0.0432528 -0.4273312]
 [ 0.0396684  0.3919179 -0.0396684 -0.3919179]
 [ 0.0432528  0.4273312 -0.0432528 -0.4273312]]


In [120]:
dir(scfdrv)

['V_es',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'acc_type',
 'add_iter_data',
 'check_convergence',
 'checkpoint_file',
 'comm',
 'comp_2e_fock',
 'comp_2e_fock_single_comm',
 'comp_2e_fock_split_comm',
 'comp_density_change',
 'comp_diis',
 'comp_energy',
 'comp_full_fock',
 'comp_gradient',
 'comp_guess_density',
 'comp_npot_mat_split_comm',
 'comp_one_ints',
 'compute',
 'compute_s2',
 'conv_thresh',
 'delete_mos',
 'den_guess',
 'den_matrices',
 'den_matrices_beta',
 'density',
 'dft',
 'diis_thresh',
 'eri_thresh',
 'eri_thresh_tight',
 'first_step',
 'fock_matrices',
 'fock_matrices_beta',
 'gen_molecular_orbitals',
 'gen_new_density',
 'get_acc_type',
 

In [144]:
# Get the ERI tensor in AO basis (chemists' notation)

nao = mo.shape[0]

pqrs = numpy.zeros((nao, nao, nao, nao))
eri_drv = vlx.ElectronRepulsionIntegralsDriver(comm)
eri_drv.compute_in_mem(molecule, basis, pqrs)
print("(pq|rs) =")
print(pqrs)

(pq|rs) =
[[[[1.0765661 0.5784704 0.2372735 0.3696176]
   [0.5784704 0.5873959 0.2587514 0.4219466]
   [0.2372735 0.2587514 0.5434929 0.3459182]
   [0.3696176 0.4219466 0.3459182 0.4529932]]

  [[0.5784704 0.3294232 0.1376483 0.2116718]
   [0.3294232 0.3613038 0.1603024 0.2618087]
   [0.1376483 0.1603024 0.3459182 0.2187686]
   [0.2116718 0.2618087 0.2187686 0.2869689]]

  [[0.2372735 0.1376483 0.0771143 0.0968359]
   [0.1376483 0.1546178 0.0968359 0.1275216]
   [0.0771143 0.0968359 0.2372735 0.1376483]
   [0.0968359 0.1275216 0.1376483 0.1546178]]

  [[0.3696176 0.2116718 0.0968359 0.1401052]
   [0.2116718 0.2342284 0.1158846 0.1774913]
   [0.0968359 0.1158846 0.2587514 0.1603024]
   [0.1401052 0.1774913 0.1603024 0.2014658]]]


 [[[0.5784704 0.3294232 0.1376483 0.2116718]
   [0.3294232 0.3613038 0.1603024 0.2618087]
   [0.1376483 0.1603024 0.3459182 0.2187686]
   [0.2116718 0.2618087 0.2187686 0.2869689]]

  [[0.5873959 0.3613038 0.1546178 0.2342284]
   [0.3613038 0.4531504 0.2014658


\begin{align}
\begin{split}
    - \sum_{\sigma \sigma'} \sum_{ijab} x_{i_{\sigma} b_{\sigma}} x_{j_{\sigma'} a_{\sigma'}}
        \langle i_{\sigma} a_{\sigma'} || j_{\sigma'} b_{\sigma} \rangle
    &= - \sum_{ijab} x_{ib} x_{ja} [ 2 (ij|ab) - 4 (ib|aj) ] \\
    &= + \sum_{ijab} x_{ib} x_{ja} [ 4 (ib|aj) - 2 (ij|ab)  ] \\
    &= \sum_{ijab} x_{ib} x_{ja} \sum_{\mu \nu \theta \varphi}
        C_{\mu i} C_{\nu b} C_{\theta a} C_{\varphi j}  [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ] \\
    &= \sum_{\mu \nu \theta \varphi} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
        \sum_{ib} C_{\mu i} x_{ib} C_{\nu b} \sum_{ja} C_{\varphi j} x_{ja} C_{\theta a} \\
    &= \sum_{\mu \nu \theta \varphi} x_{\mu \nu} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]       \\
    &= \sum_{\mu \nu} x_{\mu \nu} \sum_{\theta \varphi} x_{\varphi \theta} [ 4 (\mu \nu | \theta \varphi)
        - 2 (\mu \varphi | \theta \nu) ]
\end{split}
\end{align}

In [161]:
# Contraction of electron repulsion integrals in AO basis
# First reference value in MO
eri_contract_mo_adcc = numpy.einsum('iajb,iajb->', adcc_DM_ovov, ovov)
print("Contraction in MO adcc:", eri_contract_mo)

eri_contract_mo_vlx = (+1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, :nvir, :nocc, :nvir])##aaaa
                        +1.0*numpy.einsum('iajb,iajb->', vlx_DM_ovov, ovov[:nocc, nvir:, nocc:, :nvir]))##bbbb
print("Contraction in MO vlx: ", eri_contract_mo_vlx)

# Now contract everything in AO
eri_contract_ao = (0.5*numpy.einsum('mn,mn->', tda_eig_vec_ao, numpy.einsum('pt,mntp->mn', tda_eig_vec_ao, 4*pqrs)
                                   - numpy.einsum('pt,mptn->mn', tda_eig_vec_ao, 2*pqrs))
                    )
print("Contraction in AO:     ", eri_contract_ao)

Contraction in MO adcc: -0.2367198457073954
Contraction in MO vlx:  -0.23671984666256207
Contraction in AO:      -0.236719846662562


In [135]:
mo.shape
print(nocc + nvir)

4
